In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import scipy.stats as st
from pprint import pprint
import time
from scipy.stats import linregress
import reverse_geocoder as rg

In [6]:

# Set base url parameters
base_url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson"
magnitude = "minmagnitude=5"
limit = "limit=19999"
year=10

# set up lists to hold reponse info
time_ls = []
lat_ls = []
lng_ls = []
depth_ls = []
mag_ls = []
sig_ls = []
place_ls = []

# iterate through years
for x in range (0,11):
    time = f"starttime=20{year}-01-01&endtime=20{year}-12-31"
    query_url = (f"{base_url}&{time}&{magnitude}&{limit}")
    response = requests.get(query_url).json()

    # Append data to appropriate list
    for i in range (0, len(response["features"])):
        mag_ls.append(response["features"][i]["properties"]["mag"])
        place_ls.append(response["features"][i]["properties"]["place"])
        time_ls.append(response["features"][i]["properties"]["time"])
        sig_ls.append(response["features"][i]["properties"]["sig"])
        lat_ls.append(response["features"][i]["geometry"]["coordinates"][1])
        lng_ls.append(response["features"][i]["geometry"]["coordinates"][0])
        depth_ls.append(response["features"][i]["geometry"]["coordinates"][2])
    # counter for next year
    
    year=year+1
    

In [7]:
# create dictionary
data = {
    "Time": time_ls,
    "Lat": lat_ls,
    "Lng": lng_ls,
    "Depth": depth_ls,
    "Mag": mag_ls,
    "Sig": sig_ls,
    "Place": place_ls
}
# create dataframe
data_df = pd.DataFrame(data)

#convert timestamp to dates
data_df["Time"] = pd.to_datetime(data_df["Time"], unit="ms")
data_df["Month"] = pd.DatetimeIndex(data_df["Time"]).month
data_df["Day"] = pd.DatetimeIndex(data_df["Time"]).day
data_df["Year"] = pd.DatetimeIndex(data_df["Time"]).year

# #add country
# data_df["Country"] = ""

# for index, row in data_df.iterrows():
#     # get lat, lng from df
#     lat = row["Lat"]
#     lng = row["Lng"]
#     coordinates = (lat,lng)
#     result = rg.search(coordinates)
#     data_df.loc[index,"Country"] = result[0]["cc"]

In [8]:
# Print dataframe

data_df= data_df[["Lat","Lng","Depth","Mag","Sig","Place","Time","Month","Day","Year"]]

data_df.head()

,Lat,Lng,Depth,Mag,Sig,Place,Time,Month,Day,Year
0,-31.8300,-178.1350,35.0,5.0,385,Kermadec Islands region,2010-12-30 23:47:03.930,12,30,2010
1,-19.9840,168.3530,20.2,5.0,385,Vanuatu,2010-12-30 21:22:30.350,12,30,2010
2,50.3800,153.9640,190.1,5.0,385,Kuril Islands,2010-12-30 19:56:36.380,12,30,2010
3,56.7391,-156.1674,58.1,5.0,385,Alaska Peninsula,2010-12-30 19:21:26.463,12,30,2010
4,9.9890,126.2490,58.5,5.0,385,"Mindanao, Philippines",2010-12-30 16:56:14.930,12,30,2010


In [9]:
data_df.to_csv("output_data/earthquake_data_2010-20.csv", index= False)